<a href="https://colab.research.google.com/github/afeld/python-public-policy/blob/master/hw_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 3

1. Complete the **Coding** exercise below.
1. **Tutorial:** Go the first third of [Time Series Analysis with Pandas](https://www.dataquest.io/blog/tutorial-time-series-analysis-with-pandas/), up until the "Visualizing time series data" section.

## Coding

We are going to look at the population count of different community districts over time.

### Step 1

Read the data from the [New York City Population By Community Districts](https://data.cityofnewyork.us/City-Government/New-York-City-Population-By-Community-Districts/xi7c-iiu2/data) data set into a DataFrame called `df`, and print it out. To get the URL:

1. Visit the page linked above.
1. Click `Export`.
1. Right-click `CSV`.
1. Click `Copy Link Address` (or `Location`, depending on your browser).

In [76]:
import pandas as pd

df = pd.read_csv('https://data.cityofnewyork.us/api/views/xi7c-iiu2/rows.csv')
df

,Borough,CD Number,CD Name,1970 Population,1980 Population,1990 Population,2000 Population,2010 Population
0,Bronx,1,"Melrose, Mott Haven, Port Morris",138557,78441,77214,82159,91497
1,Bronx,2,"Hunts Point, Longwood",99493,34399,39443,46824,52246
2,Bronx,3,"Morrisania, Crotona Park East",150636,53635,57162,68574,79762
3,Bronx,4,"Highbridge, Concourse Village",144207,114312,119962,139563,146441
4,Bronx,5,"University Hts., Fordham, Mt. Hope",121807,107995,118435,128313,128200
5,Bronx,6,"East Tremont, Belmont",114137,65016,68061,75688,83268
6,Bronx,7,"Bedford Park, Norwood, Fordham",113764,116827,128588,141411,139286
7,Bronx,8,"Riverdale, Kingsbridge, Marble Hill",103543,98275,97030,101332,101731
8,Bronx,9,"Soundview, Parkchester",166442,167627,155970,167859,172298
9,Bronx,10,"Throgs Nk., Co-op City, Pelham Bay",84948,106516,108093,115948,120392


## Step 2

Prepare the data. Use the following function to [reshape](https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html) the DataFrame to have one row per community district per Census year.

In [82]:
# turn the population columns into rows
populations = pd.melt(df, id_vars=['Borough', 'CD Number', 'CD Name'], var_name='year', value_name='population')

# turn the years into numbers
populations.year = populations.year.str.replace(' Population', '').astype(int)

populations

,Borough,CD Number,CD Name,year,population
0,Bronx,1,"Melrose, Mott Haven, Port Morris",1970,138557
1,Bronx,2,"Hunts Point, Longwood",1970,99493
2,Bronx,3,"Morrisania, Crotona Park East",1970,150636
3,Bronx,4,"Highbridge, Concourse Village",1970,144207
4,Bronx,5,"University Hts., Fordham, Mt. Hope",1970,121807
...,...,...,...,...,...
290,Queens,13,"Queens Village, Rosedale",2010,188593
291,Queens,14,"The Rockaways, Broad Channel",2010,114978
292,Staten Island,1,"Stapleton, Port Richmond",2010,175756
293,Staten Island,2,"New Springville, South Beach",2010,132003


## Step 3

Create a new `manhattan` DataFrame that has only the populations for that borough.

In [90]:
manhattan = populations[populations.Borough == 'Manhattan']

## Step 4

Create a line chart of the population over time for each community district in Manhattan. There should be one line for each.

See the Plotly [Line Plot with column encoding color](https://plotly.com/python/line-charts/#line-plot-with-column-encoding-color) examples.

In [91]:
import plotly.express as px

fig = px.line(manhattan, x='year', y='population', color='CD Name')
fig.show()

## Step 5

We are going to do some mapping, but to do so, we need `borocd`s. Create a `borocd` column on the `populations` DataFrame. (See [Lecture 2](https://colab.research.google.com/github/afeld/python-public-policy/blob/master/lecture_2.ipynb#scrollTo=Pq24B8pRC_FM).)

In [92]:
def recode_borocd_pop(row):
  if row.Borough == 'Manhattan':
    return str(100 + row['CD Number'])
  elif row.Borough == 'Bronx':
    return str(200 + row['CD Number'])
  elif row.Borough == 'Brooklyn':
    return str(300 + row['CD Number'])
  elif row.Borough == 'Queens':
    return str(400 + row['CD Number'])
  elif row.Borough == 'Staten Island':
    return str(500 + row['CD Number'])
  else:
    return 'Invalid BoroCD'

populations['borocd'] = populations.apply(recode_borocd_pop, axis=1)
populations

,Borough,CD Number,CD Name,year,population,borocd
0,Bronx,1,"Melrose, Mott Haven, Port Morris",1970,138557,201
1,Bronx,2,"Hunts Point, Longwood",1970,99493,202
2,Bronx,3,"Morrisania, Crotona Park East",1970,150636,203
3,Bronx,4,"Highbridge, Concourse Village",1970,144207,204
4,Bronx,5,"University Hts., Fordham, Mt. Hope",1970,121807,205
...,...,...,...,...,...,...
290,Queens,13,"Queens Village, Rosedale",2010,188593,413
291,Queens,14,"The Rockaways, Broad Channel",2010,114978,414
292,Staten Island,1,"Stapleton, Port Richmond",2010,175756,501
293,Staten Island,2,"New Springville, South Beach",2010,132003,502


## Step 6

Create a `change` column on the `populations` DataFrame with the difference between the 2000 and 2010 population counts, then print the DataFrame.

In [68]:
df['change'] = df['2010 Population'] - df['2000 Population']
df

,Borough,CD Number,CD Name,1970 Population,1980 Population,1990 Population,2000 Population,2010 Population,change
0,Bronx,1,"Melrose, Mott Haven, Port Morris",138557,78441,77214,82159,91497,9338
1,Bronx,2,"Hunts Point, Longwood",99493,34399,39443,46824,52246,5422
2,Bronx,3,"Morrisania, Crotona Park East",150636,53635,57162,68574,79762,11188
3,Bronx,4,"Highbridge, Concourse Village",144207,114312,119962,139563,146441,6878
4,Bronx,5,"University Hts., Fordham, Mt. Hope",121807,107995,118435,128313,128200,-113
5,Bronx,6,"East Tremont, Belmont",114137,65016,68061,75688,83268,7580
6,Bronx,7,"Bedford Park, Norwood, Fordham",113764,116827,128588,141411,139286,-2125
7,Bronx,8,"Riverdale, Kingsbridge, Marble Hill",103543,98275,97030,101332,101731,399
8,Bronx,9,"Soundview, Parkchester",166442,167627,155970,167859,172298,4439
9,Bronx,10,"Throgs Nk., Co-op City, Pelham Bay",84948,106516,108093,115948,120392,4444


## Step 7

Let make a [choropleth map](https://www.data-to-viz.com/graph/choropleth.html) showing the population change for each community district that we just calculated. Adapt [the `.choropleth_mapbox()` example in Lecture 3](https://colab.research.google.com/github/afeld/python-public-policy/blob/master/lecture_3.ipynb#scrollTo=5lLS8zmn6oHP).

In [70]:
geojson = 'https://data.cityofnewyork.us/api/geospatial/yfnk-k7r4?method=export&format=GeoJSON'

fig = px.choropleth_mapbox(df,
                           geojson=geojson,
                           locations='borocd',
                           featureidkey='properties.boro_cd',
                           color='change',
                           hover_data=['CD Name'],
                           center = {'lat': 40.73, 'lon': -73.98},
                           zoom=9,
                           mapbox_style='carto-positron')

fig.update_layout(height=700)
fig.show()

## Step 8

***Analysis: Washington Heights and Inwood (the tall skinny community district at the top of Manhattan) are "up and coming" neighborhoods. Why might you guess the population decreased?***

YOUR ANSWER HERE

Then, read the first three paragraphs of the `Demographics` section of [An Economic Snapshot of Washington Heights and Inwood from June 2015](https://www.osc.state.ny.us/osdc/rpt2-2016.pdf#page=2).